# SNPs in 70 possible PFAS associated genes

### Import dependencies

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

### construct gene profile array

scrape bookmarked gnomAD pages for gene names and IDs

In [ ]:
soup = BeautifulSoup(open('gnomad bookmarks.html'))
links = soup.find_all('a')

make 2D array of gene profiles with [gene name, gene ID]

In [ ]:
gene_profiles = []
for link in links:
    gene_info = [[link.text[:-9], link.get('href')[39:54]]]
    gene_profiles += gene_info

make array of corresponding CSV filenames

In [ ]:
gene_files = os.listdir('geneCSVs')

add corresponding CSV filename to gene profile [gene name, gene ID, corresponding file]

In [ ]:
for gene_profile in gene_profiles:
    for gene_file in gene_files:
        if gene_profile[1] in gene_file[14:29]:
            gene_profile += [str(gene_file)]

### read data into dataframe(s)

loop over profiles, read corresponding CSV into DF, load DFs into array

In [ ]:
gene_variant_tables = []
for gene_profile in gene_profiles:
    table = pd.read_csv('geneCSVs/' + gene_profile[2])
    table['gene'] = gene_profile[0]
    table['geneID'] = gene_profile[1]
    table.set_index('gene', inplace = True)
    gene_variant_tables += [table]

### concatenate dataframes, keeping gene names as index

In [ ]:
frames = gene_variant_tables

In [ ]:
complete_table = pd.concat(frames)

### write text file list of rsIDs (column index 2) for batch querying

make rsID array from column in main data frame

In [ ]:
rsIDlist = []
for x in range(len(complete_table.values)):
    rsIDlist += [complete_table.values[x][2]]

write rsID array to lines in text file

In [ ]:
with open('listrsIDs.txt', 'w') as IDfile:
    for ID in rsIDlist:
        IDfile.write("%s\n" % ID)

(use text file for PolyPhen-2 batch query at http://genetics.bwh.harvard.edu/pph2/bgi.shtml and SIFT prediction query at https://sift.bii.a-star.edu.sg/www/SIFT_dbSNP.html)

### prepare main dataframe for adding PolyPhen-2 and SIFT annotations

make columns for PolyPhen-2 score, PolyPhen-2 prediction, SIFT score, and SIFT prediction

In [ ]:
complete_table['SIFT_prediction'] = ''

In [ ]:
complete_table['PPH2_prediction'] = ''

### prepare array with rsIDs, referenceNTs, alternateNTs, SIFT predictions, and PolyPhen-2 predictons - SNPs prediction profiles

(obtain SIFT results in html file - scrape)

scrape rsID, reference nucleotide, alternate nucleotide, and SIFT prediction into SIFT profiles 2D array

In [ ]:
soup = BeautifulSoup(open('SIFT-scores.html'))

In [ ]:
rows = soup.find_all('tr')

In [ ]:
SIFT_profiles = []
for row in rows[1:]:
    profile = [row.find_all('td')[0].text, \
    row.find_all('td')[4].text, \
    row.find_all('td')[5].text, \
    row.find_all('td')[15].text]
    # rsID, referenceNT, alternateNT, SIFTprediction
    SIFT_profiles += [profile]

(obtain PolyPhen-2 results in tab separated text file)

add PolyPhen-2 predictions into prediction profiles array

In [ ]:
pph2_data = pd.read_csv('pph2-full.txt', sep='\t')

clean the spaces (there are so many wow)

In [ ]:
for row in range(len(pph2_data.values)):
    for column in range(len(pph2_data.columns)):
        pph2_data.iloc[row,column] = str(pph2_data.iloc[row,column]).strip()

rsID, reference nucleotide, alternate nucleotide, and PolyPhenol-2 prediction into PPH2 profiles 2D array

In [ ]:
pph2_data.to_csv('pph2_complete.csv')

In [ ]:
PPH2_profiles = []
for row in range(len(pph2_data.values)):
    profile = [pph2_data.values[row][0], \
    pph2_data.iloc[row,9], \
    pph2_data.iloc[row,10], \
    pph2_data.iloc[row,11]]    
    # rsID, referenceNT, alternateNT, PPH2 prediction
    PPH2_profiles += [profile]

take off last few lines which are not SNP data

In [ ]:
PPH2_profiles = PPH2_profiles[:-5]

make into pandas dataframe

In [ ]:
PPH2_df = pd.DataFrame.from_records(PPH2_profiles)

In [ ]:
SIFT_df = pd.DataFrame.from_records(SIFT_profiles)

In [ ]:
PPH2_df.to_csv('pph2.csv')

In [ ]:
SIFT_df.to_csv('sift.csv')

### append main dataframe with predictions

matching requirement for both SIFT and PolyPhenol-2: rsID, refNT, and altNT (rsID alone not unique since >1 altNT)

In [ ]:
PPH2_profiles[0]

In [ ]:
complete_table.columns

first making a tinier table with only necessary data 

In [ ]:
truncated_df = complete_table
column_list = ['Source',
       'Filters - exomes', 'Filters - genomes', 'Consequence',
       'Protein Consequence', 'Transcript Consequence', 'Annotation', 'Flags',
       'Allele Count', 'Allele Number', 'Homozygote Count',
       'Hemizygote Count', 'Allele Count African', 'Allele Number African',
       'Homozygote Count African', 'Hemizygote Count African',
       'Allele Count Latino', 'Allele Number Latino',
       'Homozygote Count Latino', 'Hemizygote Count Latino',
       'Allele Count Ashkenazi Jewish', 'Allele Number Ashkenazi Jewish',
       'Homozygote Count Ashkenazi Jewish',
       'Hemizygote Count Ashkenazi Jewish', 'Allele Count East Asian',
       'Allele Number East Asian', 'Homozygote Count East Asian',
       'Hemizygote Count East Asian', 'Allele Count European (Finnish)',
       'Allele Number European (Finnish)',
       'Homozygote Count European (Finnish)',
       'Hemizygote Count European (Finnish)',
       'Allele Count European (non-Finnish)',
       'Allele Number European (non-Finnish)',
       'Homozygote Count European (non-Finnish)',
       'Hemizygote Count European (non-Finnish)', 'Allele Count Other',
       'Allele Number Other', 'Homozygote Count Other',
       'Hemizygote Count Other', 'Allele Count South Asian',
       'Allele Number South Asian', 'Homozygote Count South Asian',
       'Hemizygote Count South Asian']
for column in column_list:
    truncated_df = truncated_df.drop(column, 1)

check for matching requirement to fill in annotation columns in tinier table (rsID, refNT, and altNT)

In [ ]:
for SIFTrow in range(len(SIFT_df.values)):
    for row in range(len(truncated_df.values)):
        if truncated_df.iloc[row,2] == SIFT_df.iloc[SIFTrow,0] and \
        truncated_df.iloc[row,3] == SIFT_df.iloc[SIFTrow,1] and \
        truncated_df.iloc[row,4] == SIFT_df.iloc[SIFTrow,2]:
            truncated_df.iloc[row,7] = SIFT_df.iloc[SIFTrow,3]
for PPH2row in range(len(PPH2_profiles)):
    for row in range(len(truncated_df.values)):
        if truncated_df.iloc[row,2] == PPH2_df.iloc[PPH2row,0] and \
        truncated_df.iloc[row,3] == PPH2_df.iloc[PPH2row,1] and \
        truncated_df.iloc[row,4] == PPH2_df.iloc[PPH2row,2]:
            truncated_df.iloc[row,8] = PPH2_df.iloc[PPH2row,3]

In [ ]:
truncated_df.to_csv('annotations_2.csv')

add annotation part of tinier dataframe to master table, write to file

In [ ]:
complete_table.iloc[:,51:] = truncated_df.iloc[:,7:]

In [ ]:
complete_table.to_csv('with_SIFT_PPH2.csv')

SIFT and PPH2 using rsIDs did not return anywhere near enough results for all the SNPs, so take two: VEP

In [ ]:
VEP_table = pd.read_csv('VEP_results.txt', sep='\t')

In [ ]:
VEP_table.to_csv('VEP_only.csv')

VEP returned about 3x as many results as original data points, must remove duplicates

In [ ]:
VEP_table = VEP_table.drop('cDNA_position', axis=1)

In [ ]:
VEP_table = VEP_table.drop('CDS_position', axis=1)

In [ ]:
VEP_table = VEP_table.drop('EXON', axis=1)

In [ ]:
VEP_table = VEP_table[VEP_table.SYMBOL != '-']

In [ ]:
VEP_table = VEP_table[VEP_table.Codons != '-']

remove duplicates 

In [ ]:
VEP_table = VEP_table.drop_duplicates(subset=['#Uploaded_variation', 'Location', 'Allele'], keep='first', inplace=False)

In [ ]:
VEP_table = VEP_table.set_index('SYMBOL')

In [ ]:
VEP_table.sort_index()

In [ ]:
VEP_table['gene'] = VEP_table.index

write cleaned vegetable to CSV for record

In [ ]:
VEP_table.to_csv('veptable.csv')

In [ ]:
complete_table.sort_index()

add columns for SIFT and PPH2 annotations and also CADD but this time from the VEP data

In [ ]:
complete_table['SIFT_from_VEP'] = ''

In [ ]:
complete_table['PPH2_from_VEP'] = ''

In [ ]:
for frame in frames:
    frame.set_index('gene')
    frame['SIFT_from_VEP'] = ''
    frame['PPH2_from_VEP'] = ''
    frame['CADD_SCALED'] = ''
    frame['CADD'] = ''

go back to use individual frames for efficiency, when gene, rsID, and alternateNT match insert annotations from VEP table

In [ ]:
for VEProw in range(len(VEP_table.values)):
    for frame in frames:
        if VEP_table.iloc[VEProw,36] == frame.iloc[0,56]: # match gene
            for framerow in range(len(frame.values)): # match rsIDs nextline
                if VEP_table.iloc[VEProw,0] == frame.iloc[framerow,2] and \
                VEP_table.iloc[VEProw,2] == frame.iloc[framerow,4]: # match alternateNT
                    frame.iloc[framerow,51] = VEP_table.iloc[VEProw,23] # SIFT
                    frame.iloc[framerow,52] = VEP_table.iloc[VEProw,24] # PPH2
                    frame.iloc[framerow,54] = VEP_table.iloc[VEProw,35] # CADD
                    frame.iloc[framerow,55] = VEP_table.iloc[VEProw,34] # CADD SCALED

within individual frames re-sort by allele frequency

In [ ]:
for x in range(len(frames)):
    frames[x] = frames[x].sort_values('Allele Frequency', ascending=False)

stick them back together to form master table

In [ ]:
complete_table_2 = pd.concat(frames)

In [ ]:
complete_table_2.to_csv('sortby_gene_annotated.csv')

dataframe SIFT direct results to compare to VEP generated

In [24]:
SIFT_scores_direct = pd.read_csv('SIFT individual proteins/SIFT-scores-SLC22A6.txt', sep='\t')

In [25]:
SIFT_df = pd.DataFrame(SIFT_scores_direct)

make dict for amino acids and alphabet abbreviations

In [74]:
AA_abb = {'ala':'a', 'cys':'c', 'asp':'d', 'glu':'e', 'phe':'f', 'gly':'g', 
          'his':'h', 'ile':'i', 'lys':'k', 'leu':'l', 'met':'m', 'asn':'n', 
          'pro':'p', 'gln':'q', 'arg':'r', 'ser':'s', 'thr':'t', 'val':'v', 
          'trp':'w', 'tyr':'y'}

In [6]:
master_table_2 = pd.read_csv('SNPs_sortby_gene_annotated.csv')

In [12]:
master_table_2['SIFT_direct'] = ''

In [69]:
SIFT_df.to_csv('sift_direct.txt', sep='\t')

In [70]:
SIFT_df = pd.read_csv('sift_direct.txt', sep='\t')

In [ ]:
for SIFTrow in range(len(SIFT_df.values)):
    SIFT_position = SIFT_df.iloc[SIFTrow,0][:-6].lower()
    #print(SIFT_position)
    if SIFT_df.iloc[SIFTrow,0] != 'pos':
        SIFT_ref_aa = SIFT_df.iloc[SIFTrow,0][-6].lower()
    #print(SIFT_df.iloc[SIFTrow,0])
    #print(SIFT_ref_aa)
    for Mrow in range(372):
        m_ref_aa = master_table_2.iloc[Mrow,9][2:5].lower()
        #print(m_ref_aa)
        m_alt_aa = master_table_2.iloc[Mrow,9][-3:].lower()
        #print(m_alt_aa)
        m_position = master_table_2.iloc[Mrow,9][5:-3]
        #print(m_position)
        if m_ref_aa in AA_abb.keys() and m_alt_aa in AA_abb.keys() \
        and AA_abb[m_ref_aa] == SIFT_ref_aa and SIFT_position == m_position:
            amino_column = AA_abb[m_alt_aa].upper()
            master_table_2.iloc[Mrow,56] = SIFT_df.loc[SIFTrow,amino_column]

In [249]:
def append_master_SIFT(csv, start_row, end_row):
    SIFT_df = pd.read_csv(str('SIFT individual proteins/' + csv), sep='\t')
    for SIFTrow in range(len(SIFT_df.values)):
        SIFT_position = SIFT_df.iloc[SIFTrow,0][:-6].lower()
        #print(SIFT_position)
        if SIFT_df.iloc[SIFTrow,0] != 'pos':
            SIFT_ref_aa = SIFT_df.iloc[SIFTrow,0][-6].lower()
        #print(SIFT_df.iloc[SIFTrow,0])
        #print(SIFT_ref_aa)
        for Mrow in range(start_row, end_row + 1):
            m_ref_aa = master_table_2.iloc[Mrow,9][2:5].lower()
            #print(m_ref_aa)
            m_alt_aa = master_table_2.iloc[Mrow,9][-3:].lower()
            #print(m_alt_aa)
            m_position = master_table_2.iloc[Mrow,9][5:-3]
            #print(m_position)
            if m_ref_aa in AA_abb.keys() and m_alt_aa in AA_abb.keys() \
            and AA_abb[m_ref_aa] == SIFT_ref_aa and SIFT_position == m_position:
                amino_column = AA_abb[m_alt_aa].upper()
                master_table_2.iloc[Mrow,56] = SIFT_df.loc[SIFTrow,amino_column]

In [251]:
append_master_SIFT('SIFT-scores-SLC22A7.txt', 6385, 6765)

In [252]:
append_master_SIFT('SIFT-scores-SLC22A8.txt',  370, 714)

In [253]:
append_master_SIFT('SIFT-scores-SLC22A9.txt', 710, 1182)

In [254]:
append_master_SIFT('SIFT-scores-SLC22A12.txt', 6760, 7149)

In [256]:
append_master_SIFT('SIFT-scores-SLC51A.txt', 23870, 24153)

In [258]:
master_table_2.to_csv('master_2.csv')